## Семинар 1
# Оптимизация гиперпараметров

Гиперпараметр (англ. hyperparameter) — параметр, который не настраивается во время обучения модели. Пример гиперпараметра — шаг градиентного спуска, он задается перед обучением. Пример параметров — веса градиентного спуска, они изменяются и настраиваются во время обучения.

Попытаемся улучшить обобщающую способность модели, настроив ее гиперпараметры.
Задача оптимизации гиперпараметров заключается в подборе таких гиперпараметров, при которых заданная модель алгоритма будет наиболее эффективна.
Наиболее часто используемый метод – это решетчатый поиск (grid search), который перебирает все возможные комбинации интересующих гиперпараметров.

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import datasets
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Задания для выполнения:

### 1. Сгенерируйте данные для задачи бинарной классификации с 4 признаками и 10000 наблюдениями. Целевая переменная должна зависеть от 2 признаков. Выведите признаки в виде датафрейма, дав названия колонкам. Выведите число объектов в каждом классе.
Указание: Для этого воспользуйтесь [make_classification](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html), задав параметры: число строк n_samples=10000, число признаков n_features=4, число признаков от которых зависит целевая переменная n_informative=2, число избыточных признаков (т.е. признаков, являющихся линейными комбинациями признаков от которых зависит целевая переменная) n_redundant=2, фиксируем воспроизводимость случайных данных random_state=42.

In [7]:
X,y = datasets.make_classification(n_samples = 10000, n_features = 4, n_informative = 2, n_redundant = 2, random_state = 42)

In [8]:
X[:5]

array([[ 1.52264971, -0.93455988, -0.46502165,  0.05887383],
       [ 1.04810323, -0.74680553,  0.43685274,  0.85962787],
       [ 1.02421601, -0.44941167, -1.62343415, -1.37865568],
       [-0.43466749,  0.2817535 ,  0.02329917, -0.13524348],
       [ 1.42116496, -0.73157419, -1.46292601, -1.05843534]])

In [9]:
df = pd.DataFrame(data = X, columns = ['X'+str(i) for i in range(1, X.shape[1] + 1)])
df[:5]

,X1,X2,X3,X4
0,1.522650,-0.934560,-0.465022,0.058874
1,1.048103,-0.746806,0.436853,0.859628
2,1.024216,-0.449412,-1.623434,-1.378656
3,-0.434667,0.281754,0.023299,-0.135243
4,1.421165,-0.731574,-1.462926,-1.058435


In [10]:
pd.Series(y).value_counts()

0    5000
1    5000
Name: count, dtype: int64

### 2. Разделите данные на обучающую и тестовую части. В тестовую часть отправьте 20% объектов и зафиксируйте способ перемешивания данных random_state=42. Выведите размеры обучающей и тестовой частей.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [12]:
X_train.shape[0], X_test.shape[0]

(8000, 2000)

### 3. Постройте модель логистической регрессии с гиперпараметрами, установленными по умолчанию. Для этого создайте класс LogisticRegression c дефолтными гиперпараметрами и обучите модель на обучающих данных. Сделайте предсказание на тестовых данных. Вычислите метрику accuracy на тестовых данных.

In [13]:
model_default = LogisticRegression()

In [14]:
model_default.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'deprecated',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [15]:
model_default.fit(X_train, y_train)
accuracy_default = model_default.score(X_test, y_test)
accuracy_default

0.8935

### 4. Напишите функцию, принимающую матрицу признаков и вектор целей и осуществляющую решетчатый поиск лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием hold-out разбиения, среди двух гиперпараметров, значения которых указанны ниже. Подберите лучшие гиперпараметры на обучающих данных. Обучите модель с лучшими гиперпараметрами на обучающих данных и вычислите метрику accuracy на тестовых данных.

Создадим диапазон вариантов для гиперпараметров в виде словаря:

In [16]:
np.logspace(-3,2,9)

array([1.00000000e-03, 4.21696503e-03, 1.77827941e-02, 7.49894209e-02,
       3.16227766e-01, 1.33352143e+00, 5.62341325e+00, 2.37137371e+01,
       1.00000000e+02])

In [17]:
param_grid = {'C': np.logspace(-3,2,9), 
              'solver': ['lbfgs','liblinear', 'saga']
}

In [18]:
def Grid(X,y):
    X_trainval, X_testval, y_trainval, y_testval = train_test_split(X, y, test_size = 0.2, random_state = 42)
    best_score, score = 0, 0
    for C in param_grid['C']:
        for solver in param_grid['solver']:
            model = LogisticRegression(C = C, solver = solver)
            model.fit(X_trainval, y_trainval)
            score = model.score(X_testval, y_testval)
            if score > best_score:
                best_score = score
                best_parameters = {'C': C, 'solver': solver}
    return  best_parameters     

In [19]:
best_parameters = Grid(X_train, y_train)
best_parameters

{'C': np.float64(0.004216965034285823), 'solver': 'lbfgs'}

In [20]:
best_model = LogisticRegression(**best_parameters)
best_model.fit(X_train, y_train)
accuracy_best = best_model.score(X_test, y_test)
accuracy_best

0.894

### 5. Напишите функцию, принимающую матрицу признаков и вектор целей и осуществляющую решетчатый поиск лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием перекрёстной проверки, для тех же гиперпараметров, что и в предыдущем задании. Подберите лучшие гиперпараметры на обучающих данных. Обучите модель с лучшими гиперпараметрами на обучающих данных и вычислите метрику accuracy на тестовых данных.

In [21]:
def GridCV(X,y):
    best_score, score = 0, 0
    for C in param_grid['C']:
        for solver in param_grid['solver']:
            model = LogisticRegression(C = C, solver = solver)
            score = np.mean(cross_val_score(model, X, y, cv = 5, scoring = 'accuracy'))
            if score > best_score:
                best_score = score
                best_parameters = {'C': C, 'solver': solver}
    return  best_parameters     

In [22]:
best_parameters = GridCV(X_train, y_train)
best_parameters

{'C': np.float64(0.01778279410038923), 'solver': 'saga'}

In [23]:
best_model = LogisticRegression(**best_parameters)
best_model.fit(X_train, y_train)
accuracy_best = best_model.score(X_test, y_test)
accuracy_best

0.894

### 6. Осуществите решетчатый поиск на обучающих данных лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием перекрёстной проверки, для тех же гиперпараметров, что и в предыдущем задании, используя класс GridSearchCV из библиотеки sklearn. Убедитесь, что результаты будут теми же, что и в предыдущем задании.
Указание: Для выведения наилучших гиперпараметров модели, используйте атрибут best_params_. Наилучшая модель выводится при помощи атрибута best_estimator_.

In [24]:
gs = GridSearchCV(
    estimator = LogisticRegression(),
    param_grid = param_grid,
    cv = 5,
    scoring = 'accuracy')

In [25]:
gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': array([1.00000000e-03, 4.21696503e-03, 1.77827941e-02, 7.49894209e-02,
       3.16227766e-01, 1.33352143e+00, 5.62341325e+00, 2.37137371e+01,
       1.00000000e+02]),
                         'solver': ['lbfgs', 'liblinear', 'saga']},
             scoring='accuracy')

In [26]:
gs.best_params_

{'C': np.float64(0.01778279410038923), 'solver': 'saga'}

In [27]:
best_model = LogisticRegression(**gs.best_params_)
best_model.fit(X_train, y_train)
accuracy_best = best_model.score(X_test, y_test)
accuracy_best

0.894

In [28]:
best_model = gs.best_estimator_
best_model.fit(X_train, y_train)
accuracy_best = best_model.score(X_test, y_test)
accuracy_best

0.894

### 7. Выведите результаты решетчатого поиска с использованием класса GridSearchCV, осуществлённого в предыдущем пункте, в виде словаря и в виде датафрейма. Оставьте в датафрейме только два столбца со значениями гиперпараметров и со значениями вычисляемой метрики. Выведите самое большое значение среди средних значений метрики accuracy перекрестной проверки.
Указание: Для выведения результатов решетчатого поиска в виде словаря используйте атрибут cv_results_. Для выведения самого большого значения среди средних значений метрики accuracy используйте атрибут best_score_.

In [29]:
gs.cv_results_

{'mean_fit_time': array([0.00232935, 0.00193582, 0.00486841, 0.00198026, 0.00206828,
        0.00440912, 0.00203247, 0.00201688, 0.00422926, 0.00243034,
        0.00201497, 0.00467515, 0.00253787, 0.00218134, 0.00415049,
        0.00247231, 0.00218353, 0.00430779, 0.00235324, 0.00218596,
        0.00432391, 0.0024395 , 0.00232024, 0.00433645, 0.00249686,
        0.00228157, 0.00410099]),
 'std_fit_time': array([2.63080010e-04, 3.21205934e-04, 5.70539089e-04, 1.54196663e-04,
        1.08280259e-04, 8.18537788e-04, 1.23741622e-04, 7.03713105e-05,
        1.12995859e-04, 2.72492039e-04, 6.40469878e-05, 6.06788333e-04,
        1.94432747e-04, 8.19057733e-05, 3.00571896e-04, 1.58525558e-04,
        1.77489628e-04, 4.35970871e-04, 2.37294235e-04, 7.19192479e-05,
        5.13520089e-04, 1.85572194e-04, 1.37484764e-04, 1.52630845e-04,
        2.50308289e-04, 2.65595155e-04, 2.74509416e-04]),
 'mean_score_time': array([0.00050631, 0.0004426 , 0.00053291, 0.00043793, 0.00040445,
        0.000452

In [30]:
pd.DataFrame(gs.cv_results_).head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002329,0.000263,0.000506,0.000092,0.001000,lbfgs,"{'C': 0.001, 'solver': 'lbfgs'}",0.88125,0.878125,0.886875,0.894375,0.895000,0.887125,0.006785,26
1,0.001936,0.000321,0.000443,0.000125,0.001000,liblinear,"{'C': 0.001, 'solver': 'liblinear'}",0.88375,0.878750,0.887500,0.891875,0.895000,0.887375,0.005761,25
2,0.004868,0.000571,0.000533,0.000217,0.001000,saga,"{'C': 0.001, 'solver': 'saga'}",0.88125,0.878125,0.886875,0.894375,0.895000,0.887125,0.006785,26
3,0.001980,0.000154,0.000438,0.000090,0.004217,lbfgs,"{'C': 0.004216965034285823, 'solver': 'lbfgs'}",0.88625,0.881875,0.891875,0.900000,0.896875,0.891375,0.006654,8
4,0.002068,0.000108,0.000404,0.000044,0.004217,liblinear,"{'C': 0.004216965034285823, 'solver': 'libline...",0.88750,0.883125,0.891875,0.900625,0.896250,0.891875,0.006187,4


In [31]:
pd.DataFrame({'params':gs.cv_results_['params'], 'mean_test_score':gs.cv_results_['mean_test_score']})

,params,mean_test_score
0,"{'C': 0.001, 'solver': 'lbfgs'}",0.887125
1,"{'C': 0.001, 'solver': 'liblinear'}",0.887375
2,"{'C': 0.001, 'solver': 'saga'}",0.887125
3,"{'C': 0.004216965034285823, 'solver': 'lbfgs'}",0.891375
4,"{'C': 0.004216965034285823, 'solver': 'libline...",0.891875
5,"{'C': 0.004216965034285823, 'solver': 'saga'}",0.891375
6,"{'C': 0.01778279410038923, 'solver': 'lbfgs'}",0.892375
7,"{'C': 0.01778279410038923, 'solver': 'liblinear'}",0.892250
8,"{'C': 0.01778279410038923, 'solver': 'saga'}",0.892500
9,"{'C': 0.07498942093324558, 'solver': 'lbfgs'}",0.891625


In [32]:
gs.best_score_

np.float64(0.8924999999999998)

In [33]:
gs.cv_results_['mean_test_score'].max()

np.float64(0.8924999999999998)

### 8. Используйте методы score и predict класса GridSearchCV, чтобы вычислить метрики качества accuracy и F1-меры на тестовых данных для модели LogisticRegression с лучшими гиперпараметрами.

In [34]:
accuracy_best = gs.score(X_test, y_test)
accuracy_best

0.894

In [38]:
y_predict = gs.predict(X_test)
F1_best = metrics.f1_score(y_test, y_predict)
F1_best

0.8922764227642277

### 9. Осуществите решетчатый поиск на обучающих данных лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием перекрёстной проверки, используя класс GridSearchCV из библиотеки sklearn, среди списка словарей гиперпараметров, указанных ниже. Обучите модель с лучшими гиперпараметрами на обучающих данных и вычислите метрики accuracy и F1-меру на тестовых данных.

In [41]:
param_grid = param_grid = [{'solver': ['liblinear'],
    'penalty': ['l1', 'l2'],
    'C': np.logspace(-3,2,9),
    'max_iter' : [1000]                        
   }, {
    'solver': ['saga'],
    'penalty': ['elasticnet'],
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
    'max_iter' : [1000]
}]

model = LogisticRegression()

gs = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5
)

gs.fit(X_train, y_train)
best_model = gs.best_estimator_
y_pred = best_model.predict(X_test)

In [42]:
accuracy_best = metrics.accuracy_score(y_test, y_pred)
accuracy_best

0.894

In [43]:
f1 = metrics.f1_score(y_test, y_pred)
f1

0.8928210313447927

### 10. Осуществите рандомизированный решетчатый поиск на обучающих данных лучших гиперпараметров модели LogisticRegression по метрике accuracy с использованием перекрёстной проверки, используя класс RandomizedSearchCV из библиотеки sklearn, для списка словарей гиперпараметров из предыдущего задания. Обучите модель с лучшими гиперпараметрами на обучающих данных и вычислите метрики accuracy и F1-меру на тестовых данных.

In [45]:
param_grid = param_grid = [{'solver': ['liblinear'],
    'penalty': ['l1', 'l2'],
    'C': np.logspace(-3,2,9),
    'max_iter' : [1000]                        
   }, {
    'solver': ['saga'],
    'penalty': ['elasticnet'],
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
    'max_iter' : [1000]
}]

model = LogisticRegression()

random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    n_iter=20, 
    scoring='accuracy',  
    cv=5, 
    random_state=42  
)

random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

In [46]:
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy

0.894

In [47]:
f1 = metrics.f1_score(y_test, y_pred)
f1

0.8928210313447927

# Задания для самостоятельного выполнения:

### 1. Загрузите встроенный датасет `load_iris`. Обозначьте признаки за `X`, а целевую переменную за `y`. Выведите первые 5 строк в виде датафрейма. 

In [49]:
iris = datasets.load_iris()
X = iris.data
y = iris.target
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


### 2. Разделите данные на обучающую и тестовую части. В тестовую часть отправьте 15% объектов и зафиксируйте способ перемешивания данных random_state=0. Выведите размеры обучающей и тестовой частей.

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 0)

In [58]:
X_train.shape[0], X_test.shape[0]

(127, 23)

### 3. Для датасета load_iris осуществите классификацию методом логистической регрессии, создав класс LogisticRegression(max_iter = 1000). Выведите значения гиперпараметров этой модели, установленные по умолчанию. Обучите эту модель, сделайте предсказание. Оцените качество модели, выведя значения метрик accuracy и f1-score.

In [53]:
model = LogisticRegression(max_iter=1000)
model.get_params()


{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 1000,
 'multi_class': 'deprecated',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [54]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy

1.0

In [56]:
f1 = metrics.f1_score(y_test, y_pred, average='weighted') 
f1

1.0

### 4. Оптимизируйте гиперпараметры, используя следующие наборы гиперпараметров: param_grid = [{'solver': ['lbfgs', 'newton-cg'],'penalty': ['l2']}, {'C': np.logspace(0,4,10), 'penalty': ['l2', 'l1'], 'solver': ['liblinear']}, {'C': np.logspace(0,4,10), 'penalty': ['l2'], 'solver': ['lbfgs', 'newton-cg']},] по метрике accuracy. Реализуйте поиск лучших гиперпараметров с перекрёстной проверкой, используя класс GridSearchCV. Выведете результаты поиска в виде датафрейма, состоящего из двух колонок: в первой колонке все перебираемые сочитания гиперпараметров, а во второй соответствующие им средние значения метрики accuracy. Выведите значения лучших гиперпараметров.

In [60]:
param_grid = [
    {
        'solver': ['lbfgs', 'newton-cg'],
        'penalty': ['l2']
    },
    {
        'C': np.logspace(0, 4, 10),
        'penalty': ['l2', 'l1'],
        'solver': ['liblinear']
    },
    {
        'C': np.logspace(0, 4, 10),
        'penalty': ['l2'],
        'solver': ['lbfgs', 'newton-cg']
    }
]

model = LogisticRegression(max_iter=1000)

grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',  
    cv=5,  
)

grid_search.fit(X_train, y_train)
results = pd.DataFrame(grid_search.cv_results_)[['params', 'mean_test_score']]
results



,params,mean_test_score
0,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.960615
1,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.960615
2,"{'C': 1.0, 'penalty': 'l2', 'solver': 'libline...",0.937538
3,"{'C': 1.0, 'penalty': 'l1', 'solver': 'libline...",0.921538
4,"{'C': 2.7825594022071245, 'penalty': 'l2', 'so...",0.937231
5,"{'C': 2.7825594022071245, 'penalty': 'l1', 'so...",0.945231
6,"{'C': 7.742636826811269, 'penalty': 'l2', 'sol...",0.952923
7,"{'C': 7.742636826811269, 'penalty': 'l1', 'sol...",0.952923
8,"{'C': 21.544346900318832, 'penalty': 'l2', 'so...",0.952923
9,"{'C': 21.544346900318832, 'penalty': 'l1', 'so...",0.968615


In [61]:
grid_search.best_params_

{'C': np.float64(21.544346900318832), 'penalty': 'l1', 'solver': 'liblinear'}

### 5. Выведите значение метрики accuracy на тестовых данных. Cделайте предсказание для лучшей модели и выведите метрику f1-score. Сравните полученные метрики с метриками, полученными без оптимизации гиперпараметров.

In [62]:
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)
accuracy_best = metrics.accuracy_score(y_test, y_pred_best)
accuracy_best

0.9565217391304348

In [63]:
f1_best = metrics.f1_score(y_test, y_pred_best, average='weighted')  # Используем 'weighted' для многоклассовой классификации
f1_best

0.9550514597773576

### 6. Осуществите рандомизированный решетчатый поиск лучших гиперпараметров по метрике accuracy для той же самой сетки гиперпараметров. 

In [ ]:
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_grid,
    n_iter=20, 
    scoring='accuracy',  
    cv=5, 
    random_state=42  
)
random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_
y_pred_best = best_model.predict(X_test)
accuracy_best = metrics.accuracy_score(y_test, y_pred_best)
accuracy_best

1.0

In [70]:
f1_best = metrics.f1_score(y_test, y_pred_best, average='weighted')
f1_best

1.0

### 7. Осуществите решетчатый поиск лучших гиперпараметров по метрике f1 для той же самой сетки гиперпараметров. 

In [71]:
grid_search_f1 = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='f1_weighted', 
    cv=5
)

grid_search_f1.fit(X_train, y_train)
best_model_f1 = grid_search_f1.best_estimator_
y_pred_best_f1 = best_model_f1.predict(X_test)
accuracy_best_f1 = metrics.accuracy_score(y_test, y_pred_best_f1)
accuracy_best_f1

0.9565217391304348

In [72]:
f1_best_f1 = metrics.f1_score(y_test, y_pred_best_f1, average='weighted')
f1_best_f1

0.9550514597773576